# Input-Output Analysis and Modelling with MARIO 

##### 12th Edition of the International School of IO Analysis

Authors: Nicolò Golinucci, Lorenzo Rinaldi & Matteo Vincenzo Rocco

Affiliated to eNextGen, Politecnico di Milano

---


## Welcome to the School!
In this module, we are going to show how to use [MARIO](https://openresearchsoftware.metajnl.com/articles/10.5334/jors.473), an open-source software based on Python, inspired from [pymrio](https://pymrio.readthedocs.io/en/latest/), designed to handle any SUT or IOT table and perform easy calculations and modelling analyses.

In particular, today we are going to navigate through the following functionalities:
- Downloading & Parsing
- Aggregation
- Navigation utilities
- SUT-to-IOT transformation
- Shock analyses


### Downloading & Parsing
MARIO allows for parsing a variety of standardly adopted databases, in particular: 
- EXIOBASE, in many different versions:
    - Monetary SUTs v3.8.2, available on [Zenodo](https://zenodo.org/records/5589597) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_monetary/tutorial_parse_exiobase_monetary.html))
    - Monetary IOTs v3.8.2, available on [Zenodo](https://zenodo.org/records/5589597) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_monetary/tutorial_parse_exiobase_monetary.html))
    - Hybrid SUT v3.3.18, available on [Zenodo](https://zenodo.org/records/7244919) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_hybrid/tutorial_parse_exiobase_hybrid.html))

- EORA, both single- and multi-regional tables (EORA26), available on its [official website](https://www.worldmrio.com) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_eora/tutorial_parse_eora.html))

- FIGARO (only SUT tables), available on [Eurostat](https://ec.europa.eu/eurostat/web/esa-supply-use-input-tables/database) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_figaro_parser/tutorial_figaro_parser.html))

- Custom databases from Excel files or txt (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_custom_database/tutorial_parse_custom_database.html))

___

#### Preliminary steps

Necessary steps before being operative with MARIO for this tutorial:
- Python installed on your computer (Anaconda distribution is recommended)
- Install a conda environment from the 'environment.yml' file in this repository (MARIO is already installed). To do so, type in a terminal (or Anaconda prompt) the following: conda env create --name mario_env --file=environment.yml
- Jupyter installed in this new environment


#### Practical example: FIGARO download & parser

In this example we will see how to parse FIGARO supply-use tables.
First we need to provide the path in your computer where you desire to store the table. Then the 'download_figaro' method will need just the type of table (SUT or IOT), the year and, of course, the path.

In [ ]:
# (Optional but recommended step) MARIO dependencies are being updated, and you might have future warnings related to them. We switch them off!
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import mario

save_path = "database/FIGARO/2021"

# Download Figaro
mario.download_figaro(
    table = "SUT",
    year = 2021,
    path = save_path

)

Once downloaded, use the 'parse_FIGARO_SUT' method, providing the path where you stored the downloaded tables. MARIO will create a Database object, which we call 'world' since FIGARO is a global MRIO table (this choice is of course arbitrary)

In [ ]:
world = mario.parse_FIGARO_SUT(save_path)

It is possible to check whether the table is balanced with the 'is_balanced' method

In [ ]:
world.is_balanced('coefficients')

#### Aggregation
For the sake of keeping things simple at the beginning, we proceed to aggregate the table into a smaller one so that we can check all the next steps in parallel with an Excel conterfactual. 
MARIO allows the user to fill an Excel template to help the user provide the information on what and how to aggregate via the "get_aggregation_excel" method

In [ ]:
path_aggr = 'database/FIGARO/Aggregations/conceptual_aggregation_blank.xlsx'

world.get_aggregation_excel(
    path = path_aggr,
)

Once the Excel template is filled, the user can pass the information to MARIO via the "aggregate" method

In [ ]:
path_aggr = 'database/FIGARO/Aggregations/conceptual_aggregation_filled.xlsx'

world.aggregate(
    io=path_aggr,
    levels = [
        "Region",
        "Activity",
        "Commodity"
    ]
)

#### Exporting table to Excel

It is possible to export the table to Excel (or to a set of txt files) adopting the "to_excel" method. By default, MARIO will export the absolute values (or "flows") matrices and not the coefficients (which can actually be requested)

In [ ]:
path_concept_table = f'database/FIGARO/conceptual_excel_table.xlsx'

world.to_excel(
    path=path_concept_table,
    flows=True,
    # coefficients=True, # Uncomment to include coefficients
    )

#### Navigating table's properties

Once parsed, it is possible to check some basic information and properties characterizing the table. Just by calling the name of the database object, MARIO returns the sets of the tables in terms of number of regions, activities, commodities and others.

In [ ]:
world

It is also useful to get a list of a given set, via the "get_index": it is enough to pass the name of the set, which must be with the FIRST CAPITAL LETTER and SINGULAR

In [ ]:
world.get_index('Activity')

In [ ]:
world.get_index('Region')

##### Navigating & Calculating table's matrices

Additionally, it is possible to go through the table's matrices and calculate new ones (i.e. coefficients).
To get a matrix from a database object, it is enough to call the attribute with the name of the matrix itself: MARIO will return the matrix, calculating it first if not available.

N.B. Terminology adopted in MARIO is available in the [documentation](https://mario-suite.readthedocs.io/en/latest/terminology.html)

In [ ]:
world.Y

____
#### Changing default terminology

MARIO has its own terminology and nomenclature, however the user can speficy a new one (from v0.3.0).
It is enough to change the name of specific matrices in the settings file.

To change the default settings, you need to follow a specific structure, that is represented as a nested python dict or a can be represented as a yaml file. To access the default settings, you can use the ‘download_settings’ function, passing the path where to store the yaml.

In [ ]:
import mario
settings = mario.download_settings()

Now lets assume we want to implement two changes on the settings:
- Changing “Region” to “Country” for indexing
- Changing the Leontief matrix name from 'w' to 'L'

Once implemented, the changes can be uploaded in MARIO using "upload_settings"

In [ ]:
mario.upload_settings('settings.yaml')

You can also reset the settings to the default

In [ ]:
mario.reset_settings()

___
##### Adding environmental extensions

A table can be environmentally extended with custom satellite accounts in case they are missing.
MARIO allows to do so via the "add_extensions" method. Before that, as for the aggregation, it is possible to call the "get_extensions_excel" to ask MARIO for a template to fill with the desired data. Accounts can be provided in the "V" (think of hybrid-units tables for instance) or in the "E" matrices.

We are going to add the extensions to the full FIGARO table. Therefore we are going to parse it back from where we downloaded at the beginning

In [1]:
import mario
world = mario.parse_FIGARO_SUT('database/FIGARO/2021')

/opt/anaconda3/envs/mario_iioa/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning:

Unable to find acceptable character detection dependency (chardet or charset_normalizer).



After re-parsing, chose a path where to print the Excel template and call the 'get_extensions_excel' method, specifying also the matrix equal to 'E'

In [ ]:
extensions_path = 'database/FIGARO/Extensions/extensions_blank.xlsx'
world.get_extensions_excel(path=extensions_path, matrix='E')

For this occasion, we already prepared the emissions accounts, taken from the Exiobase database after aggregating it according to FIGARO resolution.
To use the 'add_extensions' method, it is also necessary to prepare a DataFrame with the units (it can be done also preparing it in Excel)

In [2]:
import pandas as pd

units = pd.DataFrame('kg',index=['CO2','CH4','N2O'],columns=['units'])
world.add_extensions(io='database/FIGARO/Extensions/extensions_filled.xlsx', matrix='E',units=units)

world.reset_to_coefficients(scenario='baseline')

Using add extensions will rewrite the new results on the baseline and delete other scenarios
baseline deleted from the database
Database: to calculate T following matrices are need.
['X'].Trying to calculate dependencies.


___
#### Switching to IOT
MARIO allows for SUT-to-IOT table transformations according to the methods described by [Eurostat manual]('https://mario-suite.readthedocs.io/en/latest/examples/tutorial_transformation_sut_to_iot/tutorial_transformation_sut_to_iot.html')

In [3]:
world.to_iot(method='D')

Database: to calculate Z following matrices are need.
['X'].Trying to calculate dependencies.
Singular matrix issue. The (Moore-Penrose) pseudo-inverse of a matrix will be used. This may raise some inconsistency in the data
baseline deleted from the database


In [ ]:
world

___
#### Performing a shock
MARIO allows to print an Excel template to provide information on desired shock files.
The method is called 'get_shock_excel'

In [ ]:
world.get_shock_excel('database/FIGARO/Shocks/shocks_blank.xlsx')

Once the shock is implemented in Excel, call the 'shock_calc' method to apply the shock

In [4]:
world.shock_calc(io='database/FIGARO/Shocks/shocks_filled.xlsx', Y=True, scenario='shock', force_rewrite=True)

____
##### Results post-processing

We may start by analysing the total CO2 emissions variations from the shock. The 'query' method works perfectly for the occasion, since it returns a list of matrices (just 'E' in this case) in a given scenarios ('shock') with the possibility to get the difference with respect to another scenario ('baseline'). The difference can be of absolute or relative 'type'

In [15]:
delta_E = world.query(
    matrices='E',
    scenarios='shock',
    base_scenario='baseline',
    type='absolute',
    ).loc['CO2']

delta_E.sum()

np.float64(9640.759277624176)

##### Calculating linkages
MARIO allows also to calculate forward and backward linkages (only for IOTs). The method to call is named "calc_linkages"

In [ ]:
linkages = world.calc_linkages()

Once calculated, the linkages can be also plotted by using the "plot_linkages" method. It is however suggested to aggregate the table before visualizing the results.

In [ ]:
world.get_aggregation_excel('database/FIGARO/Aggregations/linkages_aggregation_blank.xlsx')

In [ ]:
world_aggr = world.aggregate('database/FIGARO/Aggregations/linkages_aggregation_filled.xlsx',inplace=False)

In [ ]:
world.plot_linkages(multi_mode=True)

____
##### Other customized plots

In [18]:
delta_E_plot = delta_E.to_frame().T.stack([0,1,2,])
delta_E_plot

delta_E_plot = delta_E_plot.to_frame()
delta_E_plot.reset_index(inplace=True)

delta_E_plot.columns = ['Satellite account','Region','Level','Item','Value']

delta_E_plot

/var/folders/7_/vg3_ld6n2pd73xyzt9dcdhlc0000gn/T/ipykernel_1317/935930095.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



,Satellite account,Region,Level,Item,Value
0,CO2,Argentina,Sector,Accommodation and food service activities,4.079103e-03
1,CO2,Argentina,Sector,Activities auxiliary to financial services and...,4.188120e-04
2,CO2,Argentina,Sector,Activities of extraterritorial organisations a...,0.000000e+00
3,CO2,Argentina,Sector,Activities of households as employers; undiffe...,7.450581e-09
4,CO2,Argentina,Sector,Activities of membership organisations,4.126728e-04
...,...,...,...,...,...
2939,CO2,United States,Sector,Warehousing and support activities for transpo...,1.040535e-01
2940,CO2,United States,Sector,"Water collection, treatment and supply",2.268553e-03
2941,CO2,United States,Sector,Water transport,1.608307e+00
2942,CO2,United States,Sector,Wholesale and retail trade and repair of motor...,1.247466e-03


In [20]:
import plotly.express as px

fig = px.bar(
    delta_E_plot,
    x = 'Region',
    y = 'Value',
    color='Item',

)

fig.show()